### Note that all casting logic is handled by the new classes (although they do fall back to the numpy logic instead of duplicating it).

In [1]:
import proto_numpy as np


arr = np.array([1, 2, 3, 4])
arr

array([1, 2, 3, 4])
    -- BasicNumericDType(int64)

In [2]:
meters = np.UnitDType("float64", "m")

In [3]:
np.can_cast(arr.dtype, meters)  # actually can cast dtype right now

False

In [4]:
np.can_cast(arr.dtype, meters, casting="unsafe")

True

In [5]:
arr.astype(meters)

UnitDType(float64, 'm') [1. 2. 3. 4.]


array([1., 2., 3., 4.])
    -- UnitDType(float64, 'm')

In [6]:
np.add(arr, arr.astype("float64"))

BasicNumericDType(float64) [1. 2. 3. 4.]
[BasicNumericDType(int64), BasicNumericDType(float64), None]
(<class 'proto_numpy.dtypes.BasicNumericDType'>, <class 'proto_numpy.dtypes.BasicNumericDType'>, None)


array([2., 4., 6., 8.])
    -- BasicNumericDType(float64)

### Python scalar discovery works:

In [7]:
np.add(np.array([1, 2, 3], dtype=np.dtype("uint8")), 128)

[BasicNumericDType(uint8), PyIntDescriptor(128, 128), None]
(<class 'proto_numpy.dtypes.BasicNumericDType'>, <class 'proto_numpy.dtypes.BasicNumericDType'>, None)


array([129, 130, 131], dtype=uint8)
    -- BasicNumericDType(uint8)

In principle this would also work for a list inside the array to use the minimal dtype:

In [20]:
int_descr = np.discover_arr_dtype([1, 2, 3, -5], default_dtype=False, scalar_only=False)
int_descr

PyIntDescriptor(-5, 3)

But it is not a valid/finalized datatype:

In [21]:
np.dtype_is_finalized(int_descr)

False

Although for most things, aas of now, it will just be converted to a default one. *Note that users should not have the opportunity to use this "dtype"! A user will never actually see the above dtype, because they are used only internally. In principle, a user may, however, see similar more abstract dtypes to cast to a certain group, say `np.blasable` for float32 or float64.*

### Unit + Unit, checks for identical units (just strings) and then reuses the normal implementation:

In [10]:
# note that the first one is promoted correctly, general numpy promotion works:
np.add(arr.astype(np.UnitDType("int8", "m")), arr.astype(meters))

UnitDType(int8, 'm') [1 2 3 4]
UnitDType(float64, 'm') [1. 2. 3. 4.]
[UnitDType(int8, 'm'), UnitDType(float64, 'm'), None]
(<class 'proto_numpy.dtypes.UnitDType'>, <class 'proto_numpy.dtypes.UnitDType'>, None)
[BasicNumericDType(int8), BasicNumericDType(float64), None]
[BasicNumericDType(int8), BasicNumericDType(float64), None]
(<class 'proto_numpy.dtypes.BasicNumericDType'>, <class 'proto_numpy.dtypes.BasicNumericDType'>, None)
here: (BasicNumericDType(float64), BasicNumericDType(float64), BasicNumericDType(float64))


array([2., 4., 6., 8.])
    -- UnitDType(float64, 'm')

### As of now, I have not implemented mixed loops:

In [8]:
# Not implemented as of now, so it raises an error:
np.add(arr, arr.astype(meters))

UnitDType(float64, 'm') [1. 2. 3. 4.]
[BasicNumericDType(int64), UnitDType(float64, 'm'), None]
(<class 'proto_numpy.dtypes.BasicNumericDType'>, <class 'proto_numpy.dtypes.UnitDType'>, None)


TypeError: no matching loop found for (BasicNumericDType(int64), UnitDType(float64, 'm'), None).